In [ ]:
!pip install pdfplumber pandas
!pip install mistralai
!pip install --upgrade openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.7/67.7 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.6/5.6 MB 27.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 24.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.8/442.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.3/160.3 kB 10.9 MB/s eta 0:00:00


**SVB Card**

In [ ]:
import pdfplumber
import pandas as pd
import re
from datetime import datetime

pdf_path = "SVB_Statement_2025-10-31 (1).pdf"

all_cardholders = {}
lines = []

# Extrai todas as linhas de texto do PDF
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        text = page.extract_text()
        if text:
            lines.extend(text.split("\n"))

def clean_amount(raw_value):
    """
    Normaliza o valor numérico:
    - remove $ e vírgulas
    - converte parênteses e traços longos (–, −) em valores negativos
    """
    value = raw_value.strip()
    value = value.replace("$", "").replace(",", "")
    # Parênteses = negativo
    if re.match(r"^\(.*\)$", value):
        value = "-" + value.strip("()")
    # Traço longo ou similar
    value = value.replace("–", "-").replace("−", "-").strip()
    try:
        return float(value)
    except ValueError:
        return None


def parse_transaction_line(line):
    """
    Detecta linhas de transações no formato: MM-DD-YY <descrição> <valor>
    Inclui negativos, com ou sem cifrão.
    Ex: '10-31-25 UBER TRIP (12.34)'
        '10-31-25 AMAZON  -$123.45'
    """
    # Casa: data + descrição + último "token" sendo o valor
    match = re.match(
        r"(\d{2}-\d{2}-\d{2})\s+(.+?)\s+(\(?-?\$?\d{1,3}(?:,\d{3})*(?:\.\d{2})?\)?)$",
        line.strip()
    )
    if match:
        date_str = match.group(1)
        desc = match.group(2).strip()
        raw_amount = match.group(3)

        # Data normalizada
        try:
            date_obj = datetime.strptime(date_str, "%m-%d-%y")
            date_fmt = date_obj.strftime("%Y-%m-%d")
        except Exception:
            date_fmt = date_str  # fallback se der erro

        # Valor normalizado
        amount = clean_amount(raw_amount)
        if amount is None:
            return None

        return {
            "Date": date_fmt,
            "Description": desc,
            "Amount": amount,
            "MCC": "",
            "Merchant ZIP": "",
            "Description / Notes": ""
        }
    return None

# Padrão para detectar o titular (nome + total da conta)
cardholder_pattern = re.compile(r"^(.*?) TOTAL FOR ACCOUNT ENDING IN \d+.*$", re.IGNORECASE)

pending_tx = []
pending_idx = []

for i, line in enumerate(lines):
    holder_match = cardholder_pattern.match(line)
    tx = parse_transaction_line(line)

    # Se for uma transação, guarda temporariamente
    if tx:
        pending_tx.append(tx)
        pending_idx.append(i)

    # Se for linha de titular, associa as transações pendentes a esse titular
    elif holder_match:
        cardholder = holder_match.group(1).strip()
        if pending_tx:
            # Para cada transação, busca MCC e ZIP nas próximas 2 linhas
            for j, tx_item in enumerate(pending_tx):
                idx = pending_idx[j]
                next_lines = lines[idx + 1: idx + 3]
                context = " ".join(next_lines)
                mcc_match = re.search(r"MCC:\s*(\d+)", context)
                zip_match = re.search(r"MERCHANT ZIP:\s*(\d+)", context)
                tx_item["MCC"] = mcc_match.group(1) if mcc_match else ""
                tx_item["Merchant ZIP"] = zip_match.group(1) if zip_match else ""

            # Armazena as transações do titular
            all_cardholders.setdefault(cardholder, []).extend(pending_tx)

            # Limpa pendências
            pending_tx = []
            pending_idx = []

# 🔹 Tratamento extra: se o PDF não tiver seções por cardholder,
# mas tiver transações pendentes, jogamos tudo em uma aba genérica
if pending_tx:
    # tenta descobrir o número da conta pra ficar mais bonitinho
    acct_match = None
    for line in lines:
        m = re.search(r"Account Number:\s+Ending in\s+(\d+)", line)
        if m:
            acct_match = m
            break

    if acct_match:
        holder_name = f"Account {acct_match.group(1)}"
    else:
        holder_name = "All Transactions"

    all_cardholders.setdefault(holder_name, []).extend(pending_tx)
    pending_tx = []
    pending_idx = []

# Exporta para Excel: uma aba por cardholder
if not all_cardholders:
    print("Nenhum cardholder/transação encontrado. Nada foi exportado.")
else:
    output_path = "extrato_svb_by_cardholder.xlsx"
    with pd.ExcelWriter(output_path, engine="openpyxl") as writer:
        for cardholder, transactions in all_cardholders.items():
            df = pd.DataFrame(transactions)

            # Mantemos só as colunas principais (pode adicionar MCC/ZIP se quiser)
            df = df[["Date", "Description", "Amount"]]

            # Ordena por data
            df = df.sort_values(by="Date", ascending=True)

            # Excel limita o nome da sheet a 31 caracteres
            sheet_name = cardholder[:31]
            df.to_excel(writer, sheet_name=sheet_name, index=False)

    print(f"✅ Exportado para {output_path} com uma aba por cardholder (incluindo negativos)!")



✅ Exportado para extrato_svb_by_cardholder.xlsx com uma aba por cardholder (incluindo negativos)!


**Amex Card**

In [ ]:
import pdfplumber
import pandas as pd
import re
from datetime import datetime

pdf_path = "Amex 2025-11-07.pdf"   # arquivo AMEX

# ---------- Utils ----------
def normalize(s: str) -> str:
    return s.replace("–", "-").replace("−", "-").replace("⧫", "").strip()

def clean_amount(token: str):
    """Normaliza $ e negativos: -$123.45, ($123.45)"""
    t = normalize(token).replace("$", "").replace(",", "").strip()
    if re.fullmatch(r"\(\s*\d+(?:\.\d{2})?\s*\)", t):
        t = "-" + t.strip("()").strip()
    try:
        return float(t)
    except ValueError:
        return None

# === padrões ===
DATE_RE = re.compile(r"^(\d{2}/\d{2}/\d{2})\s+(.*)$")
AMOUNT_TOKEN_RE = re.compile(
    r"(?:-?\$\s*\d{1,3}(?:,\d{3})*(?:\.\d{2})?)|(?:\(\s*\d{1,3}(?:,\d{3})*(?:\.\d{2})?\s*\))"
)
PAGE_FOOTER_RE = re.compile(r"\bp\.\s*\d+/\d+\s*$", re.IGNORECASE)

# palavras que indicam início e fim de seções que devemos ignorar
FEES_START = ("FEES",)
INTEREST_START = ("INTEREST CHARGED",)
SECTION_END = (
    "TOTAL FEES FOR THIS PERIOD",
    "TOTAL INTEREST CHARGED FOR THIS PERIOD",
    "ABOUT TRAILING INTEREST",
    "IMPORTANT NOTICES",
)

# também ignorar linhas de cabeçalho / navegação
SKIP_PREFIXES = (
    "FOREIGN", "SPEND", "AMOUNT", "DETAIL", "CONTINUED ON NEXT PAGE",
)

def is_page_footer(line: str) -> bool:
    L = normalize(line)
    return bool(PAGE_FOOTER_RE.search(L))

def is_cardholder_header(lines, i):
    """
    Detecta headers do cardholder em dois formatos:
      A) <NAME IN UPPERCASE>, próxima(s) linha(s) contêm 'Card Ending'
      B) Topo de página: linhas próximas contêm 'Account Ending' ou 'Closing Date'
    """
    line = normalize(lines[i])
    if not line or line != line.upper():
        return None
    if not re.fullmatch(r"[A-Z .'\-]+", line):
        return None

    lookahead = " ".join(normalize(lines[i+k]) for k in range(1, 4) if i+k < len(lines))
    if ("CARD ENDING" in lookahead.upper()) or ("ACCOUNT ENDING" in lookahead.upper()) or ("CLOSING DATE" in lookahead.upper()):
        return line  # nome (ex.: 'KELLI SPANGLER' / 'SCOTT SOBEL')
    return None

def extract_amount_and_clean(desc_block: str):
    """
    Pega o ÚLTIMO token monetário no bloco (só $ ou parênteses),
    remove-o do texto e retorna (amount_float, descricao_limpa).
    """
    block = normalize(desc_block)
    matches = list(AMOUNT_TOKEN_RE.finditer(block))
    amount = None
    if matches:
        m = matches[-1]
        amt_raw = block[m.start():m.end()]
        amount = clean_amount(amt_raw)
        block = (block[:m.start()] + block[m.end():]).strip()
        block = re.sub(r"\s{2,}", " ", block).strip(" -|,")
    return amount, block

def parse_amex(pdf_path):
    lines = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            t = page.extract_text()
            if t:
                lines.extend(t.split("\n"))

    all_cardholders = {}
    current_holder = None

    # estado de seção a ignorar
    skip_mode = None  # None | 'fees' | 'interest'

    i, N = 0, len(lines)
    while i < N:
        raw = lines[i]
        line = normalize(raw)
        upper = line.upper()

        # --- detectar mudança de cardholder (fecha skip_mode também) ---
        holder = is_cardholder_header(lines, i)
        if holder:
            current_holder = holder.title()
            all_cardholders.setdefault(current_holder, [])
            skip_mode = None
            i += 1
            continue

        # --- entrar/sair de seções Fees/Interest ---
        if skip_mode is None and upper.startswith(FEES_START):
            skip_mode = 'fees'
            i += 1
            continue
        if skip_mode is None and upper.startswith(INTEREST_START):
            skip_mode = 'interest'
            i += 1
            continue

        if skip_mode is not None:
            # enquanto em fees/interest, ignorar tudo até marcador de término,
            # novo cardholder, topo de página ou fim
            if is_cardholder_header(lines, i) or is_page_footer(line) or any(upper.startswith(x) for x in SECTION_END):
                skip_mode = None
                # não avançamos aqui; deixa a iteração reprocessar esta linha (pode ser header)
            else:
                i += 1
            continue

        # ignorar cabeçalhos avulsos
        if upper.startswith(SKIP_PREFIXES) or is_page_footer(line):
            i += 1
            continue

        # --- transação: começa com MM/DD/YY ---
        m = DATE_RE.match(line)
        if m and current_holder:
            date_s, first_desc = m.group(1), m.group(2).strip()
            try:
                date_fmt = datetime.strptime(date_s, "%m/%d/%y").strftime("%Y-%m-%d")
            except Exception:
                date_fmt = date_s

            # acumula linhas do bloco até próxima data, novo holder, ou início de seção
            block_lines = [first_desc] if first_desc else []
            j = i + 1
            while j < N:
                nxt = normalize(lines[j])
                up = nxt.upper()

                if is_cardholder_header(lines, j):
                    break
                if DATE_RE.match(nxt):
                    break
                if up.startswith(FEES_START) or up.startswith(INTEREST_START):
                    break
                if any(up.startswith(x) for x in SECTION_END):
                    break
                if up.startswith(SKIP_PREFIXES) or is_page_footer(nxt):
                    j += 1
                    continue

                block_lines.append(nxt)
                j += 1

            block_text = " ".join([b for b in block_lines if b]).strip()
            amount, description = extract_amount_and_clean(block_text)

            all_cardholders[current_holder].append({
                "Date": date_fmt,
                "Description": description,
                "Amount": amount,
            })

            i = j
            continue

        i += 1

    return all_cardholders

# ---------- Rodar e exportar ----------
data = parse_amex(pdf_path)
# ---------- Exportar com tolerância a vazios ----------
out_path = "amex_by_cardholder.xlsx"

written_any = False
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    for holder, txs in data.items():
        # Constrói DF e garante as colunas mesmo se vazio
        df = pd.DataFrame(txs if txs else [])
        for col in ["Date", "Description", "Amount"]:
            if col not in df.columns:
                df[col] = pd.Series(dtype="object")

        # Ordena e reindexa colunas no padrão
        df = df[["Date", "Description", "Amount"]]
        df["Amount"] = pd.to_numeric(df["Amount"], errors="coerce")

        # Se não tiver nenhuma linha, pula a aba (não cria planilha vazia)
        if df.empty:
            print(f"[Info] {holder}: 0 transações (aba não criada).")
            continue

        sheet = (holder or "Unassigned")[:31]
        df.sort_values(["Date", "Description"], na_position="last").to_excel(
            writer, index=False, sheet_name=sheet
        )
        written_any = True
        print(f"[OK] {holder}: {len(df)} transações.")

    # Garante pelo menos UMA planilha visível
    if not written_any:
        pd.DataFrame(columns=["Date", "Description", "Amount"]).to_excel(
            writer, index=False, sheet_name="Summary"
        )
        print("[Warn] Nenhuma transação válida encontrada. Criada aba 'Summary' vazia.")

print(f"✅ Exportado: {out_path}")




[Info] Valor Capital Group: 0 transações (aba não criada).
[OK] Scott Sobel: 2 transações.
[Info] Telecommunications: 0 transações (aba não criada).
[OK] John Douglas Smith: 9 transações.
[OK] Kelli Spangler: 7 transações.
✅ Exportado: amex_by_cardholder.xlsx


**Bradesco**

In [ ]:
import pdfplumber
import pandas as pd
import re
from datetime import datetime, timedelta
import requests
import math

pdf_path = "Bradesco_03122025_104350.PDF"
out_path = "bradesco_card_statement_with_fx.xlsx"

# ---------- Helpers ----------
def normalize(s: str) -> str:
    return s.replace("\u00a0", " ").strip() if s else ""

def parse_brl_number(tok: str):
    """
    Converte:
      '1.234,56'  -> 1234.56
      '0,00'      -> 0.0
      '(123,45)'  -> -123.45
      '-1.234,56' -> -1234.56
    """
    if tok is None:
        return None
    t = normalize(tok)
    negative = False
    if t.startswith("(") and t.endswith(")"):
        negative = True
        t = t[1:-1]
    t = t.replace("R$", "").replace("$", "").replace(" ", "")
    # primeiro tira separador de milhar
    t = t.replace(".", "")
    # depois converte vírgula em ponto
    t = t.replace(",", ".")
    if t.startswith("-"):
        negative = True
        t = t[1:]
    try:
        val = float(t)
        return -val if negative else val
    except ValueError:
        return None

# ---------- Lê PDF e coleta linhas ----------
lines = []
with pdfplumber.open(pdf_path) as pdf:
    for page in pdf.pages:
        txt = page.extract_text()
        if txt:
            lines.extend([normalize(l) for l in txt.split("\n")])

# ---------- Descobre ano e nome do titular ----------
year = None
holder = "Cardholder"

for l in lines:
    # ex: "Mês: Setembro/2025"
    m_month = re.search(r"M[eê]s:\s*\w+\/(\d{4})", l, flags=re.IGNORECASE)
    if m_month:
        year = int(m_month.group(1))

    m_name = re.search(r"^Nome:\s*(.+)$", l, flags=re.IGNORECASE)
    if m_name:
        holder = m_name.group(1).strip()

if year is None:
    m_any = re.search(r"(20\d{2})", " ".join(lines))
    year = int(m_any.group(1)) if m_any else datetime.now().year

# ---------- Parse das transações ----------
# padrão de dinheiro BR (ex.: 0,00 | 1.399,76 | (123,45))
MONEY_BR = r"\(?-?\d{1,3}(?:\.\d{3})*,\d{2}\)?"

# dd/mm  DESCRIÇÃO  US$  R$
row_re = re.compile(
    rf"^(\d{{2}}/\d{{2}})\s+(.+?)\s+({MONEY_BR})\s+({MONEY_BR})\s*$"
)

txs = []
for l in lines:
    if l.upper().startswith(("DATA HISTÓRICO", "DATA HISTORICO")):
        continue
    if l.upper().startswith("TOTAL:"):
        continue

    m = row_re.match(l)
    if not m:
        # Fallback: pega os DOIS últimos tokens monetários da linha
        money_matches = list(re.finditer(MONEY_BR, l))
        if len(money_matches) < 2:
            continue
        m_usd, m_brl = money_matches[-2], money_matches[-1]

        # data dd/mm no começo
        m_date = re.match(r"^(\d{2}/\d{2})\s+", l)
        if not m_date:
            continue
        ddmm = m_date.group(1)

        # descrição = entre a data e o primeiro valor monetário
        desc = l[m_date.end(): m_usd.start()].strip()
        usd_raw = l[m_usd.start(): m_usd.end()]
        brl_raw = l[m_brl.start(): m_brl.end()]
    else:
        ddmm, desc, usd_raw, brl_raw = m.groups()

    day, month = ddmm.split("/")
    try:
        date_fmt = datetime(year=int(year), month=int(month), day=int(day)).strftime("%Y-%m-%d")
    except ValueError:
        date_fmt = f"{day}/{month}/{year}"

    amount_usd = parse_brl_number(usd_raw)
    amount_brl = parse_brl_number(brl_raw)

    txs.append({
        "Date": date_fmt,
        "Description": desc.strip(),
        "Amount_USD": amount_usd,
        "Amount_BRL": amount_brl,
    })

df = pd.DataFrame(txs)

# Garante colunas mesmo se vazio
for col in ["Date", "Description", "Amount_USD", "Amount_BRL"]:
    if col not in df.columns:
        df[col] = pd.Series(dtype="object")

# ---------- Função para buscar PTAX por data ----------
def get_cotacao_dolar_ptax(data_iso: str, cache: dict):
    """
    data_iso no formato YYYY-MM-DD.
    Usa a API PTAX do Bacen, com fallback para dia útil anterior.
    Retorna a cotacaoVenda (fechamento) como float.
    """
    if data_iso in cache:
        return cache[data_iso]

    # tenta até 7 dias pra trás (fim de semana/feriado)
    dt = datetime.strptime(data_iso, "%Y-%m-%d")
    for _ in range(7):
        data_bcb = dt.strftime("%m-%d-%Y")  # MM-DD-YYYY
        url = (
            "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/"
            f"CotacaoDolarDia(dataCotacao=@dataCotacao)?"
            f"@dataCotacao='{data_bcb}'&$top=100&$format=json"
        )

        try:
            resp = requests.get(url, timeout=10)
            resp.raise_for_status()
            data = resp.json()
        except Exception as e:
            print(f"[WARN] Erro ao buscar PTAX para {data_iso} ({data_bcb}): {e}")
            cache[data_iso] = None
            return None

        valores = data.get("value", [])
        if valores:
            ultimo = valores[-1]
            rate = float(ultimo["cotacaoVenda"])
            cache[data_iso] = rate
            return rate

        # se não achou, vai um dia pra trás
        dt = dt - timedelta(days=1)

    # se não encontrou nada em 7 dias, cacheia como None
    cache[data_iso] = None
    return None

# ---------- Aplica câmbio e converte BRL -> USD ----------
if not df.empty:
    fx_cache = {}

    def fx_for_row(date_str):
        if not isinstance(date_str, str):
            return None
        return get_cotacao_dolar_ptax(date_str, fx_cache)

    df["FX_BRLUSD"] = df["Date"].apply(fx_for_row)

    def convert_brl_to_usd(row):
        brl = row["Amount_BRL"]
        fx = row["FX_BRLUSD"]
        if brl is None or pd.isna(brl) or fx is None or pd.isna(fx) or fx == 0:
            return None
        return brl / fx

    df["Final Amount"] = df.apply(convert_brl_to_usd, axis=1).round(2)
    df = df[['Date','Description','Amount_BRL','FX_BRLUSD','Final Amount']]

    # Ordena por data e descrição
    df = df.sort_values(["Date", "Description"], na_position="last")



# ---------- Exporta para Excel ----------
with pd.ExcelWriter(out_path, engine="openpyxl") as writer:
    sheet = holder[:31] if holder else "Bradesco"
    df.to_excel(writer, index=False, sheet_name=sheet)

print(f"✅ Exportado: {out_path} (aba: {sheet}, {len(df)} linhas)")


✅ Exportado: bradesco_card_statement_with_fx.xlsx (aba: FELIPE M SANTOS - AMEX, 12 linhas)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')